In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as m
from scipy.interpolate import interp1d
from random import random
import scipy.optimize

def lummass (M): 
    """gets luminosity from Mass (in SunMass) """
    return pow(10, (-0.705)*np.log10(M)**2 + 4.655*np.log10(M) - 0.025)

def get_parameters(mode): 
    """gets cluster's info"""
    if (mode == 'IC2714'):
        return 10.484, 0.340,1.87
    elif (mode == 'NGC1912'):
        return 10.294, 0.253,1.82
    elif (mode == 'NGC2099'):
        return 10.74, 0.301,2.16
    elif (mode == 'NGC6834'):
        return 11.588, 0.706,0.3
    elif (mode == 'NGC7142'):
        return 10.2, 0.25,2.6
    else:
        raise ValueError 

def to_lf (a, b, c, step, alpha):
    """gets number of stars with definite magnitude"""
    func = interp1d(a, b)
    return func(c) * step * alpha

def to_mf (a, b, c, d): 
    """gets mass of a starS with definite magnitude"""
    func = interp1d(a, b)
    return func(c)*d

def neyman(data_q): 
    """gets q randomly from definite distribution"""
    iter = True
    while iter == True:
        q, Fr = random(), random()
        F = interp1d(data_q['q'], data_q['F'])
        if Fr <= F(q) and q != 0:
            iter = False
            return q
        else:
            iter = True
            
def func(x,q,lf):
    """equation to determine Mass (x)"""
    return np.log10(lummass(x))*np.log(10) + np.log(1 + 
                   pow(m.e, np.log(10)*(-0.705*np.log10(q)*np.log10(q) +
                   4.655*np.log10(q) - 1.41*np.log10(x)*np.log10(q))))-np.log(lf)    

In [28]:
in_file_lf = input("Input the Luminosity Function \n") #example "IC2714_density.txt"
in_file_is = input("Input the isochrone \n")           #example "iso_4e8.txt"
in_file_q = input("Input the distribution of q \n")    #example "func_flat.txt"

data_lf = pd.read_csv(in_file_lf, delimiter='\s+', header=None, names = ['J, mag','F','F_low', 'F_high'])
data_is = pd.read_csv("iso_4e8.txt", delimiter='\s+')
data_q = pd.read_csv(in_file_q, delimiter=' ', header=None, names = ['q','F'])
iso_young = pd.read_csv("iso_4e7.txt", delimiter='\s+')

In [29]:
dist_mod, EBV, Jlim = get_parameters (in_file_lf.split('_')[0])

data_lf['Jabs, mag'] = data_lf['J, mag']- 2.43 * 0.37 * EBV - dist_mod

step = (data_lf['Jabs, mag'].iloc[-1] - data_lf['Jabs, mag'].iloc[0]) / 30
main_table = pd.DataFrame(np.linspace(data_lf['Jabs, mag'].iloc[0], data_lf['Jabs, mag'].iloc[-1], num = 31, dtype=None)+step/2)
main_table.rename(columns={0 : 'Jabs'}, inplace = True)
main_table = main_table.drop(main_table.index[30])

results = pd.DataFrame(data = {'ALPHA': [0], 'MASS CLUSTER NEW': [0], 'RATIO': [0], 'RATIO SD': [0]})

#getting the mass of the cluster without any binary
main_table['Num_all_ss'] = np.round(to_lf(data_lf['Jabs, mag'], data_lf['F'], main_table['Jabs'], step, 1))
main_table['mass_all_ss'] = to_mf(data_is['Jmag'], data_is['Mass'], main_table['Jabs'], main_table['Num_all_ss'])
MASS_CLUSTER_OLD = main_table['mass_all_ss'].sum()

In [ ]:
algo = input("print 's' for same components of star system, 'q' if components are different")
if algo !='s' and algo!='q':
    print ("try this again, wrong symbol")

In [30]:
#for same masses
if algo == 's':
    for ALPHA in np.arange (0., 1., 0.1):
        print (ALPHA)
        MASS_BINARIES = 0
        
    #getting the mass of binaries in the cluster
        main_table['Num_tri'] = np.round(to_lf(data_lf['Jabs, mag'], data_lf['F'], main_table['Jabs'], step, ALPHA*8/45))
        main_table['Num_cua'] = np.round(to_lf(data_lf['Jabs, mag'], data_lf['F'], main_table['Jabs'], step, ALPHA*4/45))
        main_table['Num_bin'] = np.round(to_lf(data_lf['Jabs, mag'], data_lf['F'], main_table['Jabs'], step, ALPHA))- main_table['Num_tri'] - main_table['Num_cua']
    
        main_table['Lum0_bin'] = lummass(to_mf(data_is['Jmag'], data_is['Mass'], main_table['Jabs'],1))
        main_table['Lum0_bin'][main_table['Jabs'] < Jlim] = pow(10,
                                                            to_mf(iso_young['Mass'], iso_young['logL'],
                                                                  to_mf(data_is['Jmag'], data_is['Mass'], main_table['Jabs'],1),1))
        
        main_table['mass_all_bs'] = to_mf(data_is['logL'], data_is['Mass'], np.log10(main_table['Lum0_bin']/2), main_table['Num_bin'])*2
        main_table['mass_all_ts'] = to_mf(data_is['logL'], data_is['Mass'], np.log10(main_table['Lum0_bin']/3), main_table['Num_tri'])*3
        main_table['mass_all_cs'] = to_mf(data_is['logL'], data_is['Mass'], np.log10(main_table['Lum0_bin']/4), main_table['Num_cua'])*4
    
        MASS_BINARIES = main_table['mass_all_bs'].sum() +main_table['mass_all_ts'].sum() + main_table['mass_all_cs'].sum()
        
    #getting the mass of single stars in the cluster
        main_table['Num_part_ss'] = np.round(to_lf(data_lf['Jabs, mag'], data_lf['F'], main_table['Jabs'], step, 1)) - np.round(to_lf(data_lf['Jabs, mag'], data_lf['F'], main_table['Jabs'], step, ALPHA))
        main_table['mass_part_ss'] = to_mf(data_is['Jmag'], data_is['Mass'], main_table['Jabs'], main_table['Num_part_ss'])
        MASS_SINGLES = main_table['mass_part_ss'].sum()
   
        MASS_CLUSTER_NEW = MASS_BINARIES + MASS_SINGLES;
        ratio = MASS_CLUSTER_NEW / MASS_CLUSTER_OLD
        results = results.append(pd.Series([ALPHA, MASS_CLUSTER_NEW, np.mean(ratio), np.std(ratio)], index =['ALPHA', 'MASS CLUSTER NEW', 'RATIO', 'RATIO SD']), ignore_index=True)

    results = results.drop(0)
    results = results.round({'ALPHA' : 1, 'MASS CLUSTER NEW' : 2, 'RATIO' : 4, 'RATIO SD' : 4})
    results.to_csv(f"alpha_{in_file_lf.split('_')[0]}_same_4.txt"  , sep='\t', index=False, mode='w')

0.0
0.1
0.2
0.30000000000000004
0.4
0.5
0.6000000000000001
0.7000000000000001
0.8
0.9


In [27]:
# for masses by q
if algo == 'q':
    for ALPHA in np.arange (0., 1., 0.1):
        print (ALPHA)
        ratio = []
        for i in range(0, 30):
            MASS_BINARIES = 0
        
        #getting the mass of binaries in the cluster
            main_table['Num_bin'] = np.round(to_lf(data_lf['Jabs, mag'], data_lf['F_low'], main_table['Jabs'], step, ALPHA))
            main_table['Lum0_bin'] = lummass(to_mf(data_is['Jmag'], data_is['Mass'], main_table['Jabs'],1))
        
            main_table['Lum0_bin'][main_table['Jabs'] < Jlim] = pow(10,to_mf(iso_young['Mass'], iso_young['logL'],to_mf(data_is['Jmag'], data_is['Mass'], main_table['Jabs'],1),1))
        
            data_binaries = main_table.loc[main_table.index.repeat(main_table['Num_bin'].astype(int))]
            data_binaries.index = pd.RangeIndex(len(data_binaries.index))
        
            for star in range (0, len(data_binaries)):
                q=neyman(data_q)
                MASS_BINARIES += scipy.optimize.brentq(func, 0.001, 10, args = (q,data_binaries['Lum0_bin'].iloc[star])) * (1 + q)
        
        #getting the mass of single stars in the cluster
            main_table['Num_part_ss'] = np.round(to_lf(data_lf['Jabs, mag'], data_lf['F'], main_table['Jabs'], step, 1)) - np.round(to_lf(data_lf['Jabs, mag'], data_lf['F'], main_table['Jabs'], step, ALPHA))
            main_table['mass_part_ss'] = to_mf(data_is['Jmag'], data_is['Mass'], main_table['Jabs'], main_table['Num_part_ss'])
            MASS_SINGLES = main_table['mass_part_ss'].sum()
   
            MASS_CLUSTER_NEW = MASS_BINARIES + MASS_SINGLES;
            ratio.append(MASS_CLUSTER_NEW / MASS_CLUSTER_OLD)
        
        results = results.append(pd.Series([ALPHA, MASS_CLUSTER_NEW, np.mean(ratio), np.std(ratio)], index =['ALPHA', 'MASS CLUSTER NEW', 'RATIO', 'RATIO SD']), ignore_index=True)

    results = results.drop(0)
    results = results.round({'ALPHA' : 1, 'MASS CLUSTER NEW' : 2, 'RATIO' : 4, 'RATIO SD' : 4})
    results.to_csv(f"alpha_{in_file_lf.split('_')[0]}_{in_file_q.split('.txt')[0]}_corrected.txt"  , sep='\t', index=False)

0.0
0.1
0.2
0.30000000000000004
0.4
0.5
0.6000000000000001
0.7000000000000001
0.8
0.9
